In [1]:
import os
os.chdir('../../Trade networks paper/Nature Communications submission/Revision/')

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import statsmodels.api as sm
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, matthews_corrcoef, r2_score

pd.options.display.float_format = "{:,.2f}".format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Pre-processing National Trade Data

1.  **Reliability index approach:** Resolve discrepancies between importer and exporter reported trade volumes. 
2.  **Re-export algorithm:** Minimize re-exports and ensure that total exports from a country do not exceed its production plus imports. 

In [3]:
P = pd.read_csv('data/demo/prod_wheat_2017.csv') # FAOSTAT production (Wheat in 2017) 
E = pd.read_csv('data/demo/trade_wheat_2017.csv') # FAOSTAT bilateral trade (Wheat in 2017) 

In [4]:
E.head() # discrepancies between imprter and exporter reported quantities

,from_iso3,to_iso3,Exporter,Importer
0,IND,AFG,"18,975.00",0.00
1,KAZ,AFG,"284,994.19","321,106.00"
2,PAK,AFG,"2,905.21","2,764.89"
3,TKM,AFG,0.00,"169,185.00"
4,UZB,AFG,0.00,"235,069.00"


In [5]:
# Calculate accuracy level
E['AL'] = 2*(E['Importer'] - E['Exporter']).abs()/(E['Importer'] + E['Exporter'])
# if one of the reports is too small and the other is 0, ignore those rows when calculating reliability
row_cond = (E['AL']==2) & (E['Importer']<10) & (E['Exporter']<10)
E.loc[row_cond, 'AL'] = -1

# function that calculates reliability index
def _calc_rel_index(g, col, ind_col):
    d = g.copy()
    d = d[d['AL']!=-1]
    if d[col].sum()>0:
        RI = d[d['AL']<=0.2][col].sum() / d[col].sum()
    else:
        RI = 0
    g[ind_col] = RI 
    return g

# calculate importer and exporter reliability
E = E.groupby('to_iso3').apply(lambda g: _calc_rel_index(g, 'Importer', 'RIM')).reset_index(drop=True)
E = E.groupby('from_iso3').apply(lambda g: _calc_rel_index(g, 'Exporter', 'RIX')).reset_index(drop=True)
E = E.fillna(0)

E[E['AL']!=-1].sample(5)

/var/folders/w4/vjtm9mrs2411x50mjmg78b7r0000gn/T/ipykernel_91416/1120331225.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  E = E.groupby('to_iso3').apply(lambda g: _calc_rel_index(g, 'Importer', 'RIM')).reset_index(drop=True)
/var/folders/w4/vjtm9mrs2411x50mjmg78b7r0000gn/T/ipykernel_91416/1120331225.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  E = E.groupby('from_iso3').apply(lambda g: _calc_

,from_iso3,to_iso3,Exporter,Importer,AL,RIM,RIX
1897,TUR,JPN,0.00,20.00,2.00,0.72,0.31
249,BGR,HRV,25.96,0.00,2.00,1.00,0.85
1704,RUS,UZB,240.00,0.00,2.00,0.00,0.71
1005,IND,SYC,154.74,114.31,0.30,0.00,0.06
1702,RUS,UKR,509.72,37.00,1.73,0.71,0.71


In [6]:
# select the more reliable partner's report
def _select_qty(row):
    if row['RIM'] >= row['RIX']:
        return row['Importer']
    else:
        return row['Exporter']
    
E['cereal_flow'] = E.apply(lambda row: _select_qty(row), axis=1)
E[E['AL']!=-1].sample(5)

,from_iso3,to_iso3,Exporter,Importer,AL,RIM,RIX,cereal_flow
560,DEU,MLI,0.00,359.40,2.00,0.53,0.61,0.00
1683,RUS,POL,"3,097.65","3,097.65",0.00,0.42,0.71,"3,097.65"
1749,SRB,HRV,"2,695.38","3,233.04",0.18,1.00,0.54,"3,233.04"
1804,SVN,HRV,635.31,527.19,0.19,1.00,0.86,527.19
343,CAN,JPN,"1,956,027.73","1,536,916.00",0.24,0.72,0.72,"1,536,916.00"


In [7]:
def re_export_algo(P, E):
    # Implements the trade matrix re-export algorithm as given in Croft et al., 2018 
    # Number of iterations
    N = 100000 

    # Number of countries
    num_ctry = len(P)

    # Pre-calculate diagonal Production matrix
    Pd = np.diagflat(P)

    # Pre-allocate Domestic Supply matrix
    D = np.zeros((num_ctry, num_ctry))

    for n in range(1,N+1):
        # STEP 1: allocate production
        # Allocate production to domestic supply
        D += Pd / N

        # STEP 2: perform trade
        # Calculate proportions of domestic supply required for each component of export iteration
        temp1 = E / N / np.tile(np.sum(D, axis=0), (num_ctry, 1)).T

        # Sum to check if greater than 1 (if domestic supply is less than desired export total)
        temp2 = np.tile(np.nansum(temp1, axis=1)[:, np.newaxis], (1, num_ctry))

        # Constrain export greater than domestic supply to be equal to domestic supply
        mask = np.tile(np.nansum(temp1, axis=1) > 1, (num_ctry, 1)).T # or np.tile(np.nansum(temp1, axis=1)[:, np.newaxis]>1, (1, num_ctry))
        temp1[mask] = temp1[mask] / temp2[mask]
        
        # Proportional change in domestic supply
        e_n = np.ones((num_ctry, 1)) - np.nansum(temp1, axis=1)[:, np.newaxis]

        # Apply to domestic supply of domestic production (non-traded component)
        e_n = np.diagflat(e_n) + temp1

        # Take care of 0/0 cases
        e_n[np.isnan(e_n)] = 0

        # Take care of x/0 cases
        e_n[np.isinf(e_n)] = 0

        # Rescale domestic supply to redistribute according to trade
        D = D.dot(e_n)
        
    return D

In [8]:
E = E[['from_iso3', 'to_iso3', 'cereal_flow']]
E = E.merge(P[['iso3']], left_on='to_iso3', right_on='iso3', how='right').drop('to_iso3', axis=1).rename(columns={'iso3': 'to_iso3'})
E = E.sort_values(by='to_iso3')

def _add_all_countries(m):
    m = m.merge(P[['iso3']], left_on='from_iso3', right_on='iso3', how='right').drop(['from_iso3', 'to_iso3'], axis=1).rename(columns={'iso3': 'from_iso3'})
    m = m.sort_values(by='from_iso3')
    return m

E = E.groupby(['to_iso3']).apply(lambda g: _add_all_countries(g)).reset_index()
E = E[['from_iso3', 'to_iso3', 'cereal_flow']].fillna(0)

E = pd.pivot(E, index=['from_iso3'], columns = 'to_iso3',values = 'cereal_flow').reset_index()
E = E.drop(['from_iso3'], axis=1).to_numpy()

iso3_codes = P.sort_values(by='iso3')['iso3'].values.tolist()
P = P[['prod']].to_numpy()

/var/folders/w4/vjtm9mrs2411x50mjmg78b7r0000gn/T/ipykernel_91416/3412635688.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  E = E.groupby(['to_iso3']).apply(lambda g: _add_all_countries(g)).reset_index()


In [9]:
D = re_export_algo(P, E)

/var/folders/w4/vjtm9mrs2411x50mjmg78b7r0000gn/T/ipykernel_91416/4210334545.py:22: RuntimeWarning: divide by zero encountered in divide
  temp1 = E / N / np.tile(np.sum(D, axis=0), (num_ctry, 1)).T
/var/folders/w4/vjtm9mrs2411x50mjmg78b7r0000gn/T/ipykernel_91416/4210334545.py:22: RuntimeWarning: invalid value encountered in divide
  temp1 = E / N / np.tile(np.sum(D, axis=0), (num_ctry, 1)).T
/var/folders/w4/vjtm9mrs2411x50mjmg78b7r0000gn/T/ipykernel_91416/4210334545.py:29: RuntimeWarning: invalid value encountered in divide
  temp1[mask] = temp1[mask] / temp2[mask]


In [10]:
# comparing before and after re-export algorithm
df_P = pd.DataFrame(P, columns=['prod'])
df_P['iso3'] = pd.Series(iso3_codes, index=df_P.index)
first_column = df_P.pop('iso3')
df_P.insert(0, 'iso3', first_column)

df_E = pd.DataFrame(E, columns = iso3_codes)
df_E['iso3'] = pd.Series(iso3_codes, index=df_E.index)
first_column = df_E.pop('iso3')
df_E.insert(0, 'iso3', first_column)

df_D = pd.DataFrame(D, columns = iso3_codes)
df_D['iso3'] = pd.Series(iso3_codes, index=df_D.index)
first_column = df_D.pop('iso3')
df_D.insert(0, 'iso3', first_column)

In [11]:
df_E.head()

,iso3,AFG,AGO,ALB,ARE,ARG,ARM,ATG,AUS,AUT,AZE,BDI,BEL,BEN,BFA,BGD,BGR,BHR,BHS,BIH,BLR,BLZ,BOL,BRA,BRB,BRN,BTN,BWA,CAF,CAN,CHE,CHL,CHN,CIV,CMR,COD,COG,COL,COM,CPV,CRI,CUB,CYP,CZE,DEU,DJI,DMA,DNK,DOM,DZA,ECU,EGY,ERI,ESP,EST,ETH,FIN,FJI,FRA,FSM,GAB,GBR,GEO,GHA,GIN,GMB,GNB,GNQ,GRC,GRD,GTM,GUY,HKG,HND,HRV,HTI,HUN,IDN,IND,IRL,IRN,IRQ,ISL,ISR,ITA,JAM,JOR,JPN,KAZ,KEN,KGZ,KHM,KNA,KOR,KWT,LAO,LBN,LBR,LBY,LCA,LKA,LSO,LTU,LUX,LVA,MAC,MAR,MDA,MDG,MDV,MEX,MHL,MKD,MLI,MLT,MMR,MNE,MNG,MOZ,MRT,MUS,MWI,MYS,NAM,NCL,NER,NGA,NIC,NIU,NLD,NOR,NPL,NRU,NZL,OMN,PAK,PAN,PER,PHL,PNG,POL,PRI,PRK,PRT,PRY,PSE,PYF,QAT,ROU,RUS,RWA,SAU,SDN,SEN,SGP,SLB,SLE,SLV,SOM,SRB,SSD,STP,SUR,SVK,SVN,SWE,SWZ,SYC,SYR,TCD,TGO,THA,TJK,TKM,TLS,TON,TTO,TUN,TUR,TUV,TWN,TZA,UGA,UKR,URY,USA,UZB,VCT,VEN,VNM,VUT,WSM,YEM,ZAF,ZMB,ZWE
0,AFG,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,AGO,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,ALB,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,ARE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.07,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.90,3.62,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,60.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0

In [12]:
df_D.head() # diagonal elements represent domestic consumption

,iso3,AFG,AGO,ALB,ARE,ARG,ARM,ATG,AUS,AUT,AZE,BDI,BEL,BEN,BFA,BGD,BGR,BHR,BHS,BIH,BLR,BLZ,BOL,BRA,BRB,BRN,BTN,BWA,CAF,CAN,CHE,CHL,CHN,CIV,CMR,COD,COG,COL,COM,CPV,CRI,CUB,CYP,CZE,DEU,DJI,DMA,DNK,DOM,DZA,ECU,EGY,ERI,ESP,EST,ETH,FIN,FJI,FRA,FSM,GAB,GBR,GEO,GHA,GIN,GMB,GNB,GNQ,GRC,GRD,GTM,GUY,HKG,HND,HRV,HTI,HUN,IDN,IND,IRL,IRN,IRQ,ISL,ISR,ITA,JAM,JOR,JPN,KAZ,KEN,KGZ,KHM,KNA,KOR,KWT,LAO,LBN,LBR,LBY,LCA,LKA,LSO,LTU,LUX,LVA,MAC,MAR,MDA,MDG,MDV,MEX,MHL,MKD,MLI,MLT,MMR,MNE,MNG,MOZ,MRT,MUS,MWI,MYS,NAM,NCL,NER,NGA,NIC,NIU,NLD,NOR,NPL,NRU,NZL,OMN,PAK,PAN,PER,PHL,PNG,POL,PRI,PRK,PRT,PRY,PSE,PYF,QAT,ROU,RUS,RWA,SAU,SDN,SEN,SGP,SLB,SLE,SLV,SOM,SRB,SSD,STP,SUR,SVK,SVN,SWE,SWZ,SYC,SYR,TCD,TGO,THA,TJK,TKM,TLS,TON,TTO,TUN,TUR,TUV,TWN,TZA,UGA,UKR,URY,USA,UZB,VCT,VEN,VNM,VUT,WSM,YEM,ZAF,ZMB,ZWE
0,AFG,"4,280,775.98",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,AGO,0.00,"2,915.74",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,ALB,0.00,0.00,"274,877.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,ARE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0

### Initial Subnational Flow Allocation

1.  **Classification model:** Train model to identify existing links. 
2.  **Regression model:** Train model to predict volume of flows on existing links.
3.  **Prediction on subnational pairs:** Use trained models for predicting subnational flows.

In [13]:
# features
X_cols = ['transport_USD_t_log',
    'time_h_log',
    'distance_km_log', 
    'border_USD_t_log', 
    'customs_cost',
    'tariff_log', 
    'from_price_log',
    'from_barley_area_log', 
    'from_maize_area_log', 
    'from_millet_area_log', 
    'from_rice_area_log',
    'from_sorghum_area_log', 
    'from_wheat_area_log', 
    'from_other_cereals_area_log',
    'from_barley_production_log', 
    'from_maize_production_log',
    'from_millet_production_log', 
    'from_rice_production_log',
    'from_sorghum_production_log', 
    'from_wheat_production_log',
    'from_other_cereals_production_log', 
    'from_buffaloes_log', 
    'from_cattle_log',
    'from_chickens_log', 
    'from_ducks_log', 
    'from_goats_log', 
    'from_horses_log', 
    'from_pigs_log',
    'from_sheep_log', 
    'from_pop_log', 
    'from_gdp_log',
    'from_area_log', 
    'from_built_volume_total_log',
    'from_region_0', 
    'from_region_1', 
    'from_region_2', 
    'from_region_3',
    'from_region_4', 
    'from_region_5',
    'to_price_log',
    'to_barley_area_log',   
    'to_maize_area_log',
    'to_millet_area_log', 
    'to_rice_area_log', 
    'to_sorghum_area_log', 
    'to_wheat_area_log',
    'to_other_cereals_area_log', 
    'to_barley_production_log', 
    'to_maize_production_log',
    'to_millet_production_log', 
    'to_rice_production_log', 
    'to_sorghum_production_log',
    'to_wheat_production_log', 
    'to_other_cereals_production_log', 
    'to_buffaloes_log',
    'to_cattle_log', 
    'to_chickens_log', 
    'to_ducks_log', 
    'to_goats_log', 
    'to_horses_log',
    'to_pigs_log', 
    'to_sheep_log', 
    'to_pop_log', 
    'to_gdp_log',
    'to_area_log', 
    'to_built_volume_total_log', 
    'to_region_0', 
    'to_region_1', 
    'to_region_2', 
    'to_region_3',
    'to_region_4', 
    'to_region_5',
    'domestic',
    'subnational'
]

# classification metrics
def class_metrics_group(g, y_col):
    accuracy = accuracy_score(g[y_col], g['pred'])
    precision = precision_score(g[y_col], g['pred'])
    recall = recall_score(g[y_col], g['pred'])
    mcc = matthews_corrcoef(g[y_col], g['pred'])
    return pd.Series(dict(accuracy=accuracy, precision=precision, recall=recall, mcc=mcc))

# regression metrics
def reg_metrics_group(g, y_col):
    r2 = r2_score(g[y_col], g['pred'])
    r2_log = r2_score(np.log(g[y_col]+1), np.log(g['pred']+1))
    return pd.Series(dict(r2=r2, r2_log=r2_log))

In [14]:
df_train = pd.read_csv('data/demo/df_train.csv')
df_train.shape

(42182, 81)

In [15]:
# classification
y_col = 'supply_cereals_all_exists'

# split into train and test
train, test = train_test_split(df_train, test_size=0.2, random_state=4)
X_train = train[X_cols]
y_train = train[y_col]
X_test = test[X_cols]
y_test = test[y_col]

# define model
model =  HistGradientBoostingClassifier(max_iter=5000, 
                                        class_weight='balanced', 
                                        min_samples_leaf=100, 
                                        max_features=0.8,
                                        categorical_features=['from_region_0','from_region_1', 'from_region_2', 
                                                              'from_region_3', 'from_region_4', 'from_region_5',
                                                              'to_region_0', 'to_region_1', 'to_region_2', 
                                                              'to_region_3', 'to_region_4', 'to_region_5'],
                                       random_state=4)

# fit model
model.fit(X_train, y_train)

# predict on train and test
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
train['pred'] = y_train_pred
test['pred'] = y_test_pred

In [16]:
# performance on train 
class_metrics_group(train, y_col)

accuracy    0.97
precision   0.92
recall      0.98
mcc         0.93
dtype: float64

In [17]:
# performance on test
class_metrics_group(test, y_col)

accuracy    0.94
precision   0.86
recall      0.92
mcc         0.85
dtype: float64

In [18]:
# train a model on all data (for making predictions on subnational pairs)
X = df_train[X_cols]
y = df_train[y_col]

# define model 
model1 =  HistGradientBoostingClassifier(max_iter=5000, 
                                        class_weight='balanced', 
                                        min_samples_leaf=100, 
                                        max_features=0.8,
                                        categorical_features=['from_region_0','from_region_1', 'from_region_2', 
                                                              'from_region_3', 'from_region_4', 'from_region_5',
                                                              'to_region_0', 'to_region_1', 'to_region_2', 
                                                              'to_region_3', 'to_region_4', 'to_region_5'],
                                       random_state=4)

# train model 
model1.fit(X, y)

HistGradientBoostingClassifier(categorical_features=['from_region_0',
                                                     'from_region_1',
                                                     'from_region_2',
                                                     'from_region_3',
                                                     'from_region_4',
                                                     'from_region_5',
                                                     'to_region_0',
                                                     'to_region_1',
                                                     'to_region_2',
                                                     'to_region_3',
                                                     'to_region_4',
                                                     'to_region_5'],
                               class_weight='balanced', max_features=0.8,
                               max_iter=5000, min_samples_leaf=100,
                               random_state=4)

In [19]:
# regression 
y_col = 'supply_cereals_all'
# only choose rows where a link exists
df_train = df_train[df_train[y_col]>=1]
print(df_train.shape)

# split into train and test
train, test = train_test_split(df_train, test_size=0.2, random_state=4)
X_train = train[X_cols]
y_train = train[y_col]
X_test = test[X_cols]
y_test = test[y_col]

# define model
model = HistGradientBoostingRegressor(l2_regularization=100, 
                                      loss='gamma',
                                      max_iter=1000,
                                      max_depth=8,
                                      max_features=0.4,
                                      min_samples_leaf=100,
                                      categorical_features=['from_region_0','from_region_1', 'from_region_2', 
                                                            'from_region_3', 'from_region_4', 'from_region_5',
                                                            'to_region_0', 'to_region_1', 'to_region_2', 
                                                            'to_region_3', 'to_region_4', 'to_region_5'],
                                     random_state=4)

# fit model
model.fit(X_train, y_train)

# predict on train and test
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
train['pred'] = y_train_pred
test['pred'] = y_test_pred

(11490, 81)


In [20]:
# performance on train 
reg_metrics_group(train, y_col)

r2       0.90
r2_log   0.90
dtype: float64

In [21]:
# performance on test 
reg_metrics_group(test, y_col)

r2       0.41
r2_log   0.72
dtype: float64

In [22]:
# train a model on all data (for making predictions on subnational pairs)
X = df_train[X_cols]
y = df_train[y_col]

# define model 
model2 = HistGradientBoostingRegressor(l2_regularization=100, 
                                      loss='gamma',
                                      max_iter=1000,
                                      max_depth=8,
                                      max_features=0.4,
                                      min_samples_leaf=100,
                                      categorical_features=['from_region_0','from_region_1', 'from_region_2', 
                                                            'from_region_3', 'from_region_4', 'from_region_5',
                                                            'to_region_0', 'to_region_1', 'to_region_2', 
                                                            'to_region_3', 'to_region_4', 'to_region_5'],
                                     random_state=4)

# train model 
model2.fit(X, y)

HistGradientBoostingRegressor(categorical_features=['from_region_0',
                                                    'from_region_1',
                                                    'from_region_2',
                                                    'from_region_3',
                                                    'from_region_4',
                                                    'from_region_5',
                                                    'to_region_0',
                                                    'to_region_1',
                                                    'to_region_2',
                                                    'to_region_3',
                                                    'to_region_4',
                                                    'to_region_5'],
                              l2_regularization=100, loss='gamma', max_depth=8,
                              max_features=0.4, max_iter=1000,
                              min_samples_leaf=100, random_state=4)

In [23]:
# predict on subnational pairs
df_pred = pd.read_csv('data/demo/df_pred.csv')
df_pred['supply_cereals_all_exists_pred'] = model1.predict_proba(df_pred[X_cols])[:,1]
df_pred['supply_cereals_all_pred'] = model2.predict(df_pred[X_cols])

In [24]:
df_pred['supply_cereals_all_exists_pred'].describe()

count   10,000.00
mean         0.10
std          0.20
min          0.00
25%          0.00
50%          0.01
75%          0.07
max          1.00
Name: supply_cereals_all_exists_pred, dtype: float64

In [25]:
df_pred[df_pred['supply_cereals_all_exists_pred']>0.5]['supply_cereals_all_pred'].describe()

count      706.00
mean       696.18
std      2,074.18
min          0.70
25%         68.38
50%        214.64
75%        601.35
max     26,976.39
Name: supply_cereals_all_pred, dtype: float64

### Harmonization of Predicted Flows

1.  **Estimating consumption:** Estimate regional demand.
2.  **Iterative harmonizing:** Ensure aggregated subnational flows match national flows, and each subnational region meets its consumption.

In [45]:
cons_country = pd.read_csv('data/demo/master_df_country.csv')
cons_admin = pd.read_csv('data/demo/master_df_admin.csv')

# model for fitting consumption
cols = ['buffaloes_log', 'cattle_log', 'chickens_log', 
        'ducks_log', 'goats_log', 'horses_log', 
        'pop_log', 'gdp_log', 'cereals_all_prod_log']

fml = "cereals_all_cons ~ " + " + ".join(cols) + " - 1 "
cons_mod = sm.GLM.from_formula(fml, family=sm.families.Gamma(link=sm.families.links.Log()), data=cons_country)
res = cons_mod.fit()
res.summary(cons_mod)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                            Generalized Linear Model Regression Results                                            
===================================================================================================================================
Dep. Variable:     <statsmodels.genmod.generalized_linear_model.GLM object at 0x13dfed9a0>   No. Observations:                  195
Model:                                                                                 GLM   Df Residuals:                      186
Model Family:                                                                        Gamma   Df Model:                            8
Link Function:                                                                         Log   Scale:                          1.0632
Method:                                                                               IRLS   Log-Likelihood:                -3009.7
Date:                                                                     Thu, 05 Jun 2025   Deviance:                       119.87
Time:                                                                             19:37:18   Pearson chi2:                     198.
No. Iterations:                                                                        100   Pseudo R-squ. (CS):             0.9787
Covariance Type:                                                                 nonrobust                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
buffaloes_log            0.0028      0.018      0.160      0.873      -0.032       0.038
cattle_log              -0.0037      0.060     -0.061      0.951      -0.122       0.115
chickens_log             0.0688      0.043      1.594      0.111      -0.016       0.153
ducks_log                0.0099      0.020      0.497      0.619      -0.029       0.049
goats_log               -0.0444      0.035     -1.262      0.207      -0.113       0.025
horses_log               0.0565      0.030      1.867      0.062      -0.003       0.116
pop_log                  0.7883      0.052     15.145      0.000       0.686       0.890
gdp_log                 -0.0093      0.032     -0.290      0.772      -0.072       0.054
cereals_all_prod_log     0.0949      0.038      2.500      0.012       0.021       0.169
========================================================================================
"""

In [46]:
# R2 while fitting
cons_country['cereals_all_cons_pred'] = res.predict(cons_country[cols]) 
print(f"R2: {r2_score(cons_country['cereals_all_cons'], cons_country['cereals_all_cons_pred'])}")
print(f"R2 on logs: {r2_score(np.log(cons_country['cereals_all_cons']+1), np.log(cons_country['cereals_all_cons_pred']+1))}")

R2: 0.897171597389441
R2 on logs: 0.8748160136043076


In [47]:
# use gamma regression model to estimate subnational consumption
cons_admin['cereals_all_cons'] = res.predict(cons_admin[cols])

cons_comp = cons_admin.groupby('iso3').sum()[['cereals_all_cons']].reset_index().rename(columns={
    'cereals_all_cons': 'cereals_all_cons_total'}).merge(cons_country[['iso3', 'cereals_all_cons']].rename(columns={
    'cereals_all_cons': 'cereals_all_cons_country'}))

# R2 between country level consumption and totalled predicted admin level consumption
print(f"R2: {r2_score(cons_comp['cereals_all_cons_country'], cons_comp['cereals_all_cons_total'])}")
print(f"R2 on logs: {r2_score(np.log(cons_comp[f'cereals_all_cons_country']+1), np.log(cons_comp['cereals_all_cons_total']+1))}")

# scale estimated subnational consumption to match national levels
cons_admin = cons_admin.merge(cons_comp)
cons_admin['cereals_all_cons'] = cons_admin['cereals_all_cons'] * cons_admin['cereals_all_cons_country'] / cons_admin['cereals_all_cons_total']

R2: 0.8880397780417469
R2 on logs: 0.8719496100942287


In [48]:
master_df_admin = cons_admin[['iso3', 'ID', 'cereals_all_prod', 'cereals_all_cons']].sort_values(by=['iso3', 'ID']).reset_index(drop=True)
df_country = pd.read_csv('data/demo/df_train.csv')
df_country = df_country[df_country['dataset']=='faostat'].sort_values(by=['from_iso3', 'to_iso3']).reset_index(drop=True)
df_ids = master_df_admin[['ID', 'iso3']].sort_values(by='ID').reset_index(drop=True).rename(columns={'ID': 'id'})

# ML predictions 
df_mat = pd.read_csv('data/demo/predictions_all.csv')

In [49]:
# harmonizing algorithm 
def probs_edit(g, dom_q=0.8, int_q=0.8, threshold=0.5, crop='cereals_all'):
    if g[f'supply_{crop}_exists_pred'].sum()==0:
        return g
    if g['from_iso3'].values[0]==g['to_iso3'].values[0]:
        if g[f'supply_{crop}_exists_pred'].quantile(dom_q)>threshold:
            return g
        g.loc[g[f'supply_{crop}_exists_pred']>=g[f'supply_{crop}_exists_pred'].quantile(dom_q), f'supply_{crop}_exists_pred'] = threshold+0.1
    else:
        if g[f'supply_{crop}_exists_pred'].quantile(int_q)>threshold:
            return g
        g.loc[g[f'supply_{crop}_exists_pred']>=g[f'supply_{crop}_exists_pred'].quantile(int_q), f'supply_{crop}_exists_pred'] = threshold+0.1
    return g


def _rescale(g, crop='cereals_all'):
    if g['from_iso3'].values[0]==g['to_iso3'].values[0]:
        return g
    if g[f'supply_{crop}_pred'].sum()==0:
        return g
    if g[f'supply_{crop}'].values[0] < g[f'supply_{crop}_pred'].sum(): 
        # if target trade is lower, then it doesn't matter that some regions have reached their outflow capacity (even in the constrained situation)
        g['scaling_factor'] = g[f'supply_{crop}'].values[0] / g[f'supply_{crop}_pred'].sum()
    else:
        # if target trade is higher, and some regions have reached their outflow capacity, we need to exclude those in the constrained situation
        target_excl = g[f'supply_{crop}'].values[0] - g[g['exclude']==1][f'supply_{crop}_pred'].sum()
        current_excl = g[g['exclude']==0][f'supply_{crop}_pred'].sum()
        if current_excl==0:
            g.loc[g['exclude']==0, f'supply_{crop}_pred'] = g[g['exclude']==0]['remaining']
            current_excl = g[g['exclude']==0][f'supply_{crop}_pred'].sum()
        g['scaling_factor'] = target_excl / current_excl
        g.loc[g['exclude']==1, 'scaling_factor'] = 1
    g[f'supply_{crop}_pred'] = g[f'supply_{crop}_pred'] * g['scaling_factor']
    return g

def int_rescale(F, df_country, df_ids, beta=0.95, constrained=False, crop='cereals_all'):
    # convert to dataframe
    df_F = pd.DataFrame(F, columns = df_ids['id'].values.tolist())
    df_F['from_id'] = pd.Series(df_ids['id'].values.tolist(), index=df_F.index)
    first_column = df_F.pop('from_id')
    df_F.insert(0, 'from_id', first_column)
    df_F = df_F.melt(id_vars=['from_id'], value_vars=df_ids['id'].values.tolist()).rename(columns={'variable': 'to_id', 'value': f'supply_{crop}_pred'})
    df_F = df_F.merge(df_ids, left_on='from_id', right_on='id').rename(columns={'iso3': 'from_iso3'}).drop('id', axis=1)
    df_F = df_F.merge(df_ids, left_on='to_id', right_on='id').rename(columns={'iso3': 'to_iso3'}).drop('id', axis=1)
    df_F = df_F.merge(df_country[['from_iso3', 'to_iso3', f'supply_{crop}']], how='left')
    
    outflow_status = df_F.groupby('from_id')[[f'supply_{crop}_pred']].sum().reset_index().merge(master_df_admin[['ID', f'{crop}_prod']], left_on='from_id', right_on='ID').drop('ID', axis=1)
    outflow_status['exclude'] = 0
    outflow_status.loc[outflow_status[f'supply_{crop}_pred'] >= beta * beta * outflow_status[f'{crop}_prod'], 'exclude'] = 1
    outflow_status['remaining'] = outflow_status[f'{crop}_prod'] - outflow_status[f'supply_{crop}_pred']
    df_F = df_F.merge(outflow_status[['from_id', 'remaining', 'exclude']])
    if not constrained:
        df_F['exclude'] = 0
    # rescale
    df_F = df_F.groupby(['from_iso3', 'to_iso3']).apply(lambda g: _rescale(g, crop)).reset_index(drop=True)

    # convert to matrix
    F = pd.pivot(df_F, index=['from_id'], columns='to_id', values = f'supply_{crop}_pred').reset_index().drop('from_id', axis=1).to_numpy()

    return F

def harmonize_algorithm(df_mat, master_df_admin, df_country, df_ids, crop, 
                        threshold=0.5, tol=0.05, max_iter=50, alpha=0.1, beta=0.95, dom_q=0.8, int_q=0.8):

    df_mat = df_mat.groupby(['from_iso3', 'to_iso3']).apply(lambda g: probs_edit(g, dom_q, int_q, threshold, crop)).reset_index(drop=True)
    df_mat = df_mat.sort_values(by=['from_id', 'to_id'])
    df_mat_probs = pd.pivot(df_mat, index=['from_id'], columns = 'to_id',values = f'supply_{crop}_exists_pred').reset_index().drop('from_id', axis=1).to_numpy()
    df_mat_trade = pd.pivot(df_mat, index=['from_id'], columns = 'to_id',values = f'supply_{crop}_pred').reset_index().drop('from_id', axis=1).to_numpy()

    F = df_mat_trade
    prob_matrix = df_mat_probs
    F = np.where(prob_matrix >= threshold, F, 0)

    n_regions = master_df_admin['ID'].nunique()
    region_to_country = master_df_admin['iso3'].values
    production = master_df_admin[f'{crop}_prod'].values
    consumption = master_df_admin[f'{crop}_cons'].values

    inflow = F.sum(axis=0)       # shape (n_regions,)
    outflow = F.sum(axis=1)      # shape (n_regions,)
    net = production + inflow - outflow  # net resources at each region
    diff = consumption - net     # positive diff indicates deficit, negative indicates surplus
    tol_threshold = tol * consumption
    
    print(f"Before balancing: Total = {F.sum()}, Sum deficits = {diff[diff > tol_threshold].sum():.2f}, " +
          f"Sum surpluses = {diff[diff < -tol_threshold].sum():.2f}")

    for iteration in range(max_iter):

        # Save F before updating, to later compute max change.
        F_old = F.copy()
        
        # Adjust international flows to match national targets.
        F = int_rescale(F, df_country, df_ids, beta, constrained=False, crop=crop)
    
        max_change = 0
        
        # Adjust flows to meet the consumption constraints.
        inflow = F.sum(axis=0)       # shape (n_regions,)
        outflow = F.sum(axis=1)      # shape (n_regions,)
        net = production + inflow - outflow  # net resources at each region
        diff = consumption - net     # positive diff indicates deficit, negative indicates surplus
        tol_threshold = tol * consumption
    
        # Diagnostic prints:
        print(f"Iteration {iteration}: Sum deficits = {diff[diff > tol_threshold].sum():.2f}, " +
              f"Sum surpluses = {diff[diff < -tol_threshold].sum():.2f}")
    
        # Initialize scaling factors as 1 for every region.
        scaling_cols = np.ones(n_regions)
        scaling_rows = np.ones(n_regions)

        # can change this if the flows seem to be reducing/increasing too much or if nothing changes 
        add_fac = (1 + iteration/(max_iter-1)) * alpha # goes from alpha to 2*alpha 
        sub_fac = (0.5 + iteration/(max_iter-1)) * alpha # goes from 0.5*alpha to 1.5*alpha 
        
        # Loop over each region and apply the appropriate correction.
        for i in range(n_regions):
            if diff[i] > tol_threshold[i]:  # Region i in deficit.
                if inflow[i] > 0:
                    scaling_cols[i] = (inflow[i] + add_fac * diff[i]) / inflow[i]
                if outflow[i] > 0:
                    scaling_rows[i] = (outflow[i] - sub_fac * diff[i]) / outflow[i]
            elif diff[i] < -tol_threshold[i]:  # Region i in surplus.
                if inflow[i] > 0:
                    scaling_cols[i] = (inflow[i] + sub_fac * diff[i]) / inflow[i]
                # Only adjust outflows if region i also has un-exported production.
                if (production[i] - outflow[i]) > 0 and outflow[i] > 0:
                    scaling_rows[i] = (outflow[i] - add_fac * diff[i]) / outflow[i]
        
        # Clip scaling factors to ensure they are not negative.
        scaling_cols = np.maximum(scaling_cols, 0)
        scaling_rows = np.maximum(scaling_rows, 0)
        
        # Update F.
        # Multiply each column i (i.e. inflows to region i) by scaling_cols[i]:
        F = F * scaling_cols[None, :]
        # Multiply each row i (i.e. outflows from region i) by scaling_rows[i]:
        F = F * scaling_rows[:, None]
    
        print("Flow sum after consumption adjustment =", F.sum())
    
        # Enforce constraint: total outflows must not exceed beta*production, 
        # also if production allows, at least 50% consumption must be met by local production (so constraint must take that into account too) 
        # need to justify the second (can do using national scale data, and/or india data?)
        for i in range(n_regions):
            current_outflow = F[i, :].sum()
            constrained_outflow = min(beta * production[i], max(0, production[i]-0.5*consumption[i])) 
            if current_outflow > constrained_outflow: 
                factor = constrained_outflow / current_outflow
                F[i, :] *= factor
        
        print("Flow sum after enforcing production constraint =", F.sum())
        
        # Track maximum change from this consumption adjustment.
        max_change = np.max(np.abs(F - F_old))
        if max_change < 1:
            print("Converged at iteration", iteration)
            break

    F = int_rescale(F, df_country, df_ids, beta, constrained=True, crop=crop)

    inflow = F.sum(axis=0)       # shape (n_regions,)
    outflow = F.sum(axis=1)      # shape (n_regions,)
    net = production + inflow - outflow  # net resources at each region
    diff = consumption - net     # positive diff indicates deficit, negative indicates surplus
    tol_threshold = tol * consumption
    
    # Diagnostic prints:
    print(f"After first loop: Total = {F.sum()}, Sum deficits = {diff[diff > tol_threshold].sum():.2f}, " +
          f"Sum surpluses = {diff[diff < -tol_threshold].sum():.2f}")

    excess_outflow = 0
    for i in range(n_regions):
        current_outflow = F[i, :].sum()
        if current_outflow > production[i]: 
            excess_outflow += current_outflow - production[i]
    print(f'Excess outflow = {excess_outflow}')

    for iteration in range(2*max_iter):

        # Save F to later compute max change.
        F_old = F.copy()
    
        # Compute the residual consumption difference.
        inflow = F.sum(axis=0)
        outflow = F.sum(axis=1)
        net = production + inflow - outflow
        diff = consumption - net  # positive indicates additional flow needed.
    
        # Diagnostic prints:
        print(f"Iteration {iteration}: Sum deficits = {diff[diff > tol_threshold].sum():.2f}, " +
              f"Sum surpluses = {diff[diff < -tol_threshold].sum():.2f}")
    
        countries = np.unique(region_to_country)
        for c in countries:
            # Get indices for regions in country c.
            idx = np.where(region_to_country == c)[0]
            if len(idx) == 0:
                continue
        
            # Compute aggregates for regions in country c.
            # Calculate inflow for each region in idx:
            inflow_c = F[:, idx].sum(axis=0)
            outflow_c = F[idx, :].sum(axis=1)
            net_c = production[idx] + inflow_c - outflow_c
            diff_c = consumption[idx] - net_c  # Positive: deficit, Negative: surplus
        
            # Identify deficit and surplus regions.
            mask_deficit = diff_c > tol_threshold[idx]  # deficit regions
    
            # Identify surplus regions with two conditions:
            # 1. Their net exceeds their consumption (net > consumption)
            # 2. Their total outflow is less than production (so they are not already exporting all their production)
            mask_surplus = (net_c > consumption[idx]) & (outflow_c < production[idx])
        
            if np.sum(mask_deficit) == 0 or np.sum(mask_surplus) == 0:
                continue
        
            deficit_regions = idx[mask_deficit]
            surplus_regions = idx[mask_surplus]
            
            # Total deficit and total available surplus (absolute values).
            total_deficit = diff_c[mask_deficit].sum()
    
            # For each surplus region, compute its available surplus in two ways:
            #   a) net surplus = net - consumption
            #   b) unused production = production - outflow
            # Use the minimum of these two as the effective surplus.
            available_surplus = np.minimum(-diff_c[mask_surplus], production[idx][mask_surplus] - outflow_c[mask_surplus]) # since diff is negative for surplus
            total_surplus = available_surplus.sum()  
        
            # Determine the reallocation amount R.
            R = alpha * min(total_deficit, total_surplus)
            if R <= 0:
                continue
        
            # Allocate extra flow from surplus to deficit regions.
            # For each deficit region, compute its share of total deficit.
            deficit_shares = diff_c[mask_deficit] / total_deficit  if total_deficit > 0 else np.ones(np.sum(mask_deficit))
            # For each surplus region, compute its share of total surplus.
            surplus_shares = available_surplus / total_surplus  if total_surplus > 0 else np.ones(np.sum(mask_surplus))
            
            # For each deficit region i and each surplus region j, allocate extra flow.
            for di, i in enumerate(deficit_regions):
                for sj, j in enumerate(surplus_regions):
                    # The extra flow contributed from donor j to recipient i:
                    delta = R * deficit_shares[di] * surplus_shares[sj]
                    F[j, i] += delta
       
        print(F.sum())
        
        # Track maximum change from this consumption adjustment.
        max_change = np.max(np.abs(F - F_old))
        if max_change < 1:
            print("Converged at iteration", iteration)
            break
    
    return F

In [50]:
F = harmonize_algorithm(df_mat, master_df_admin, df_country, df_ids, 'cereals_all')
df_F = pd.DataFrame(F, columns = df_ids['id'].values.tolist())
df_F['from_id'] = pd.Series(df_ids['id'].values.tolist(), index=df_F.index)
first_column = df_F.pop('from_id')
df_F.insert(0, 'from_id', first_column)
df_F = df_F.melt(id_vars=['from_id'], value_vars=df_ids['id'].values.tolist()).rename(columns={'variable': 'to_id', 'value': 'supply_cereals_all_bal'})
df_F = df_F.merge(df_ids, left_on='from_id', right_on='id').rename(columns={'iso3': 'from_iso3'}).drop('id', axis=1)
df_F = df_F.merge(df_ids, left_on='to_id', right_on='id').rename(columns={'iso3': 'to_iso3'}).drop('id', axis=1)

/var/folders/w4/vjtm9mrs2411x50mjmg78b7r0000gn/T/ipykernel_91416/1396850515.py:65: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_mat = df_mat.groupby(['from_iso3', 'to_iso3']).apply(lambda g: probs_edit(g, dom_q, int_q, threshold, crop)).reset_index(drop=True)


Before balancing: Total = 2194159810.54658, Sum deficits = 1105076410.32, Sum surpluses = -1104851398.13


/var/folders/w4/vjtm9mrs2411x50mjmg78b7r0000gn/T/ipykernel_91416/1396850515.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_F = df_F.groupby(['from_iso3', 'to_iso3']).apply(lambda g: _rescale(g, crop)).reset_index(drop=True)


Iteration 0: Sum deficits = 762165890.46, Sum surpluses = -764910509.92
Flow sum after consumption adjustment = 1674620521.2537103
Flow sum after enforcing production constraint = 908435459.9328493


/var/folders/w4/vjtm9mrs2411x50mjmg78b7r0000gn/T/ipykernel_91416/1396850515.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_F = df_F.groupby(['from_iso3', 'to_iso3']).apply(lambda g: _rescale(g, crop)).reset_index(drop=True)


...

Iteration 48: Sum deficits = 4388774.17, Sum surpluses = -11163928.30
Flow sum after consumption adjustment = 1185478887.1291878
Flow sum after enforcing production constraint = 1184532432.3177319


/var/folders/w4/vjtm9mrs2411x50mjmg78b7r0000gn/T/ipykernel_91416/1396850515.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_F = df_F.groupby(['from_iso3', 'to_iso3']).apply(lambda g: _rescale(g, crop)).reset_index(drop=True)


Iteration 49: Sum deficits = 4056696.99, Sum surpluses = -10976165.13
Flow sum after consumption adjustment = 1185458146.3022926
Flow sum after enforcing production constraint = 1184592861.6212342


/var/folders/w4/vjtm9mrs2411x50mjmg78b7r0000gn/T/ipykernel_91416/1396850515.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_F = df_F.groupby(['from_iso3', 'to_iso3']).apply(lambda g: _rescale(g, crop)).reset_index(drop=True)


After first loop: Total = 1183689544.3368561, Sum deficits = 4387574.10, Sum surpluses = -10364237.75
Excess outflow = 47127.700636383204
Iteration 0: Sum deficits = 4387574.10, Sum surpluses = -10364237.75
1184105499.3246515
Iteration 1: Sum deficits = 3087744.15, Sum surpluses = -9921106.61
1184392117.3747346
Iteration 2: Sum deficits = 2624953.20, Sum surpluses = -9642763.10
1184632925.9328399
Iteration 3: Sum deficits = 2251603.83, Sum surpluses = -9414428.79
1184836442.4736376
Iteration 4: Sum deficits = 1921986.85, Sum surpluses = -9223572.19
1185006997.3163064
Iteration 5: Sum deficits = 1702221.31, Sum surpluses = -9069501.12
1185155575.6043818
Iteration 6: Sum deficits = 1506733.78, Sum surpluses = -8934258.79
1185284605.1397283
Iteration 7: Sum deficits = 1332790.76, Sum surpluses = -8811189.66
1185396240.3734777
Iteration 8: Sum deficits = 1101048.92, Sum surpluses = -8708986.19
1185484702.060228
Iteration 9: Sum deficits = 984479.87, Sum surpluses = -8620528.19
1185561557.8

In [52]:
df_F['supply_cereals_all_bal'].sum()

1186009611.0141122

In [51]:
df_F['supply_cereals_all_bal'].describe()

count   12,503,296.00
mean            94.86
std          6,721.02
min              0.00
25%              0.00
50%              0.00
75%              0.00
max      5,147,526.71
Name: supply_cereals_all_bal, dtype: float64

In [53]:
df_F[df_F['from_iso3']!=df_F['to_iso3']]['supply_cereals_all_bal'].sum()

455377537.2854958

In [54]:
df_F[(df_F['from_iso3']==df_F['to_iso3']) 
    & (df_F['from_id']!=df_F['to_id'])]['supply_cereals_all_bal'].sum()

730632073.7286148